In [2]:
import duckdb
import os
import glob
from datetime import datetime, timezone

In [3]:

# ---- CONFIG ----
## vigyazz a pathra
DB_PATH = "../databases/currency_rates.duckdb"
JSON_FOLDER = "../jsons/fx_jsons"
SCHEMA_NAME = "analytics"
TABLE_NAME = "mart_currency_fx_sek"

# 1) DB csatlakozás -- ha nincs meg db file akkor letrehoz egyet
con = duckdb.connect(DB_PATH)

In [4]:
# 8) Teszt folyt
df = con.execute("""
    select *
    from analytics.mart_currency_fx_sek
    where rate_date = '2025-12-01'
""").df()

df.tail(50)

,fx_rate_sk,rate_date,loaded_at_utc,currency,rate,rolling_max_1y,rolling_min_1y,drawdown_1y,rolling_max_3y,rolling_min_3y,drawdown_3y,rolling_max_5y,rolling_min_5y,drawdown_5y,rolling_max_10y,rolling_min_10y,drawdown_10y
0,ee7098c8836c6d0e25917e0cbbd75ed0,2025-12-01,2025-12-06T21:16:32.514778+00:00,CAD,6.751988,7.824681,6.702307,-0.137091,8.300787,6.702307,-0.186585,8.322812,6.399822,-0.188737,8.322812,5.867812,-0.188737
1,941b2616999d0ee1670b72f638542d3a,2025-12-01,2025-12-06T21:16:32.514778+00:00,CHF,11.746179,12.438303,11.242307,-0.055645,12.495817,11.001372,-0.059991,12.495817,9.114243,-0.059991,12.495817,8.212075,-0.059991
2,2da7ee41ac4542bbf3a87378d5657586,2025-12-01,2025-12-06T21:16:32.514778+00:00,DKK,1.469173,1.552952,1.441204,-0.053948,1.601782,1.441204,-0.082789,1.601782,1.328276,-0.082789,1.601782,1.227885,-0.082789
3,995f1bbe80acb6fab130692866a00172,2025-12-01,2025-12-06T21:16:32.514778+00:00,EUR,10.971734,11.585879,10.753386,-0.053008,11.953151,10.753386,-0.082105,11.953151,9.883077,-0.082105,11.953151,9.150561,-0.082105
4,21c9b8f20e871f8c408ca5e892374dcd,2025-12-01,2025-12-06T21:16:32.514778+00:00,GBP,12.486761,14.011284,12.372055,-0.108807,14.011284,12.372055,-0.108807,14.011284,11.077463,-0.108807,14.011284,10.254943,-0.108807
5,5b03b49d46b4c577d4fed735d2be477b,2025-12-01,2025-12-06T21:16:32.514778+00:00,HUF,0.028819,0.028867,0.026490,-0.001664,0.031799,0.025963,-0.093723,0.031799,0.025438,-0.093723,0.033919,0.025438,-0.150368
6,913f6ad8ab189efcccb1b732b2c698b5,2025-12-01,2025-12-06T21:16:32.514778+00:00,NOK,0.933064,0.994823,0.911932,-0.062080,1.086242,0.911932,-0.141016,1.106216,0.911932,-0.156526,1.110266,0.887539,-0.159603
7,1a2948fc0ecd21dabab96e6b6e7ff760,2025-12-01,2025-12-06T21:16:32.514778+00:00,USD,9.453290,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000


In [5]:
df = con.execute(
    """
    select
        *
    from analytics.int_currency_rates
    where rate_date in ('2025-12-05')--, '2025-12-04')
    and currency_pair in ('USDCAD', 'USDEUR', 'USDGBP', 'USDCHF', 'USDDKK', 'USDNOK', 'USDHUF', 'USDSEK')
    """
).df()

df.head(20)

# USDCAD, USDEUR, USDGBP, USDCHF, USDDKK, USDNOK, USDHUF, USDSEK

,filename,loaded_at_utc,rate_date,currency_pair,rate
0,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-06T21:16:32.514778+00:00,2025-12-05,USDEUR,0.858704
1,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-06T21:16:32.514778+00:00,2025-12-05,USDDKK,6.414904
2,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-06T21:16:32.514778+00:00,2025-12-05,USDCHF,0.804333
3,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-06T21:16:32.514778+00:00,2025-12-05,USDCAD,1.382750
4,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-06T21:16:32.514778+00:00,2025-12-05,USDNOK,10.105104
5,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-06T21:16:32.514778+00:00,2025-12-05,USDHUF,328.020388
6,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-06T21:16:32.514778+00:00,2025-12-05,USDGBP,0.749625
7,/Users/zsoltnagy/sec_edgar/jsons/fx_jsons/usd_...,2025-12-06T21:16:32.514778+00:00,2025-12-05,USDSEK,9.400050


In [6]:
# futasd a checkpointot, h a currency_rates.duckdb.wal file (Write ahead log) ami a valtoztatasokat tartalömazza eltunjön
# vegul zard be a connectiont!
con.execute("CHECKPOINT;")
con.close()